# Basic Data Prep and DT/Entropy Assignment

1. In class we found that

`classTree = DecisionTreeClassifier(criterion='entropy',max_depth=1,random_state=0)
classTree.fit(X,y)`

resulted in an initial split on the Income predictor field at 87.5 (thousand dollars). With 79.2% of our samples satisfying this condition (income $\leq$ 87.5) resulting in a group with an entropy of 0.949, the other 29.2% of our samples didn't satisfy this condition and resulted in a pure group (entropy = 0). The weighted average of this split at the value is then (0.792)(0.949) + (0.292)(0) = 0.751.

What if, instead of using this split, we split the data using an income level of 65 (income $\leq$ 65)? Let's investigate why this should be a worse rule. Using "income $\leq$ 65," what percentage of the data (only 24 total data points) would satisfy this condition? Use Shannon's model of entropy (eqn 4.1 in text and the one shown in class) to compute the entropy of this subset of the data (i.e., find the proportion of non-owners and proportion of owners and use the equation to compute). Repeat this for subset of the data that didn't satisfy the "income $\leq$ 65" condition. Now compute the weighted average of the resulting entropy as we did for the first rule above. Compare the weighted average entropy of this split with the weighted average entropy above (0.751) and discuss. 

(I'm guiding you to compute weighted average entropy for just one other potential split value and to explain/show why that's a worse choice than what the algorithm found (87.5) - this is how the algorithm is working recursively)

Proportion of income $\leq$ 65: 13
Percentage of owners = 4 / 13 = 0.3077
Percentage of non-owners = 9 / 13 = 0.6923
Entropy of this subset = -0.3077 * log2(0.3077) - 0.6923 * log2(0.6923) = 0.4791 + 0.5158 = 0.8904

Proportion of income $\gt$ 65: 11
Percentage of owners = 8 / 11 = 0.7273
Percentage of non-owners = 3 / 11 = 0.2727
Entropy of this subset = -0.7273 * log2(0.7273) - 0.2727 * log2(0.2727) = 0.4791 + 0.5158 = 0.8453

Weighted average Entropy = 13 / 24 * 0.8904 + 11 / 24 * 0.8453 = 0.4081 + 0.3874 = 0.7955

which is greater than 0.751 so that it is worse





In [1]:
import os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import MinMaxScaler, StandardScaler

pd.set_option("display.max_rows", 25)

In [2]:
os.chdir('C:\\Users\\Yuqin Yang\\Documents\\ANOP330\\Data')

In [3]:
bank_df = pd.read_csv('UniversalBank.csv')
bank_df

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,0,1,0


**Download the data UniversalBankData.csv on the Moodle site and read in the data**

2. Check the data types and change "ZIP Code" to an object and verify that it's type was correctly changed.

In [4]:
bank_df.dtypes

ID                      int64
Age                     int64
Experience              int64
Income                  int64
ZIP Code                int64
Family                  int64
CCAvg                 float64
Education               int64
Mortgage                int64
Personal Loan           int64
Securities Account      int64
CD Account              int64
Online                  int64
CreditCard              int64
dtype: object

In [5]:
bank_df['ZIP Code'] = bank_df['ZIP Code'].astype('object')

In [6]:
bank_df.dtypes

ID                      int64
Age                     int64
Experience              int64
Income                  int64
ZIP Code               object
Family                  int64
CCAvg                 float64
Education               int64
Mortgage                int64
Personal Loan           int64
Securities Account      int64
CD Account              int64
Online                  int64
CreditCard              int64
dtype: object

3.  Check for missing data (I'm not saying there is any, but verify!). If it's a numerical column, replace with the median value. If it's categorical drop the record.

In [7]:
bank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   object 
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(12), object(1)
memory usage: 547.0+ KB


In [8]:
bank_df.describe()

,ID,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000
mean,2500.500000,45.338400,20.104600,73.774200,2.396400,1.937938,1.881000,56.498800,0.096000,0.104400,0.06040,0.596800,0.294000
std,1443.520003,11.463166,11.467954,46.033729,1.147663,1.747659,0.839869,101.713802,0.294621,0.305809,0.23825,0.490589,0.455637
min,1.000000,23.000000,-3.000000,8.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,1250.750000,35.000000,10.000000,39.000000,1.000000,0.700000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,2500.500000,45.000000,20.000000,64.000000,2.000000,1.500000,2.000000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000
75%,3750.250000,55.000000,30.000000,98.000000,3.000000,2.500000,3.000000,101.000000,0.000000,0.000000,0.00000,1.000000,1.000000
max,5000.000000,67.000000,43.000000,224.000000,4.000000,10.000000,3.000000,635.000000,1.000000,1.000000,1.00000,1.000000,1.000000


In [9]:
bank_df.count()

ID                    5000
Age                   5000
Experience            5000
Income                5000
ZIP Code              5000
Family                5000
CCAvg                 5000
Education             5000
Mortgage              5000
Personal Loan         5000
Securities Account    5000
CD Account            5000
Online                5000
CreditCard            5000
dtype: int64

4. Use the `s.strip().replace(...)` (see overview slides for syntax) and a list comprehension to remove all the empty spaces in the column titles.

In [10]:
bank_df.columns = [s.strip().replace(' ', '_') for s in bank_df.columns]
bank_df.columns

Index(['ID', 'Age', 'Experience', 'Income', 'ZIP_Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Personal_Loan', 'Securities_Account',
       'CD_Account', 'Online', 'CreditCard'],
      dtype='object')

5. Rescale the Age, Experience, Income, Family, Education, and Mortgage fields by transforming the values into their z-scores. Use the sklearn function StandardScaler to accomplish this (make sure you use `pd.DataFrame(..)` to create the new dataframe. Display the head of the scaled dataframe with all the original fields .

(You're going to need to do this on a subset of the dataframe and then overwrite the fields after you do it, so use your previous notes on how to get a subset - I'd create a list of the fields you want to scale and use that)

In [11]:
scaler = StandardScaler()
stand_df = pd.DataFrame(scaler.fit_transform(bank_df), 
   index = bank_df.index, 
   columns = bank_df.columns)
stand_df.head()

,ID,Age,Experience,Income,ZIP_Code,Family,CCAvg,Education,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard
0,-1.731704,-1.774417,-1.666078,-0.538229,-0.964114,1.397414,-0.193385,-1.049078,-0.555524,-0.325875,2.928915,-0.25354,-1.216618,-0.645314
1,-1.731012,-0.029524,-0.096330,-0.864109,-1.443932,0.525991,-0.250611,-1.049078,-0.555524,-0.325875,2.928915,-0.25354,-1.216618,-0.645314
2,-1.730319,-0.552992,-0.445163,-1.363793,0.738814,-1.216855,-0.536736,-1.049078,-0.555524,-0.325875,-0.341423,-0.25354,-1.216618,-0.645314
3,-1.729626,-0.901970,-0.968413,0.569765,0.452243,-1.216855,0.436091,0.141703,-0.555524,-0.325875,-0.341423,-0.25354,-1.216618,-0.645314
4,-1.728933,-0.901970,-1.055621,-0.625130,-0.859007,1.397414,-0.536736,0.141703,-0.555524,-0.325875,-0.341423,-0.25354,-1.216618,1.549632


6. Create a 60/40 train to validation sample from this DataFrame. Use `train_test_split(..)` and a `random_state=1.` Display the shape of each of these new DataFrames.

In [12]:
trainData, validData = train_test_split(stand_df, test_size=0.40, random_state=1)
print('Training   : ', trainData.shape)
print('Validation : ', validData.shape)

Training   :  (3000, 14)
Validation :  (2000, 14)


7. The bank is interested in identifying customers that might be interested in taking out a personal loan. The provided data is labeled for those that have a personal loan, a securities account, a CD account, an online account or a credit card. What would your analytics model be to help answer this question? 

We can use these historical data to predict whether the customer is interested in taking out a personal loan. For example, we can use the label "have a personal loan" and the label "credit card" to check whether the customer has enough money and whether probably needs to take another loan.